<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/lec_title.png?raw=true" alt="2019年度ゲノム情報解析入門" height="100px" align="middle">

In [0]:
# /// 実習前に、このセルを実行してください。 ///

# サンプルデータをダウンロード
!wget -O AB_classification.csv https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2019/dataset/AB_classification.csv
# 「分類」アルゴリズムの描画関数などをダウンロード
!wget -O classification.py https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2019/modules/classification.py
# 決定木描画用のライブラリをインストール
!pip install -q dtreeviz

# 機械学習 - 分類 -

　今回、教師あり学習（Supervised learning）のひとつ、**分類（Classification）**の各種アルゴリズムを勉強します。

## この実習で使用するデータセット

　各種の分類アルゴリズムを試すために、次のような[データセット](https://github.com/CropEvol/lecture/blob/master/textbook_2019/dataset/AB_classification.csv)を使います。

- サンプル数（計84サンプル）
  - 種A 46サンプル
  - 種B 38サンプル

- 説明変数（分類では、**特徴量（Feature）**と呼ぶことが多いです）
  - x1, x2

- 目的変数（分類では、**クラス（Class）**や**ラベル（Label）**と呼ぶことが多いです）
  - species


<small>*※ このデータセットは、以下の論文のデータセットを一部改変したデータです:  
Ohta, A., Yamane, K., & Kawahara, T. (2017). [Relationship between spike morphology and habitat of four Aegilops species of section Sitopsis.](https://link.springer.com/article/10.1007/s10722-016-0408-x) Genetic resources and crop evolution, 64(5), 889-899.*</small>




In [0]:
# 読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("AB_classification.csv", sep=",", header=0)

# 説明変数
x = df.loc[:,["x1","x2"]]
cols = x.columns  # 列名
x = np.array(x)    # Numpy配列に変換

# ラベル変数
y = np.array(df.loc[:,"species"])  # 種名

# グラフ
a = df[df["species"] == "A"]
b = df[df["species"] == "B"]
plt.scatter(a["x1"], a["x2"], color="red", label="A")
plt.scatter(b["x1"], b["x2"], color="blue", label="B")
plt.xlabel("x1")
plt.ylabel("x2")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show

## 今回の実習内容
1. 前処理
1. ロジスティック回帰 Logistic regression
1. サポートベクトルマシン Support vector machine (SVM)
1. 決定木 Decision tree
1. ランダムフォレスト Random Forest
1. ニューラルネットワーク Neural network
1. 評価

---

## 前処理

　「回帰」のときと同じく、「分類」でも前処理は必要です。ここでは、次の3つの前処理をおこないます。
1. データの分割: トレーニングデータとテストデータを作る
  - トレーニングデータ 75%、テストデータ 25%
  - ラベルの割合が、分割後のデータでも維持されるように分割する（オプション`stratify`）

2. スケーリング: 説明変数の尺度を揃える
  - トレーニングデータを「標準化」（平均0、標準偏差1となるように変換）
  - テストデータも、トレーニングデータの変換水準で「標準化」

3. ラベルの数値変換: ラベル変数を整数値に変換する
  - `A` → `0`
  - `B` → `1`


In [0]:
# (1) データの分割
from sklearn.model_selection import train_test_split
splited_dataset = train_test_split(x, y, test_size=0.25, stratify=y, random_state=0)
x_train_raw, x_test_raw = splited_dataset[0], splited_dataset[1]
y_train_raw, y_test_raw = splited_dataset[2], splited_dataset[3]

# 確認
print("original: ", np.unique(y, return_counts=True))      # 元のデータセット内の各クラスのサンプル数
print("training: ", np.unique(y_train_raw, return_counts=True))# トレーニングデータ内の各クラスのサンプル数
print("test: ", np.unique(y_test_raw, return_counts=True))    # テストデータ内の各クラスのサンプル数

# (2) スケーリング
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train_ss = ss.fit_transform(x_train_raw) # トレーニングデータ
x_test_ss = ss.transform(x_test_raw) # テストデータ

# 確認
#print(x_test_ss)

# (3) ラベルの数値変換
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_le = le.fit_transform(y_train_raw) # トレーニングデータ
y_test_le = le.transform(y_test_raw) # テストデータ

# 確認
print(y_test_raw)
print(y_test_le)

### 実習1

　下記コードの`train_test_split()`のオプション`stratify=b`を削除すると、分割後の各ラベルのサンプル数がどうなるか確認してください。

In [0]:
import numpy as np
# データ
a = np.ones(10)
b = np.array(["rice"]*5 + ["wheat"]*5)

# (1) データの分割
from sklearn.model_selection import train_test_split
a_train, a_test, b_train, b_test = train_test_split(a, b, test_size=0.4, stratify=b, random_state=30)
#a_train, a_test, b_train, b_test = train_test_split(a, b, test_size=0.4, random_state=30)

# 確認
print("original: ", np.unique(b, return_counts=True))      # 元のデータセット内の各クラスのサンプル数
print("training: ", np.unique(b_train, return_counts=True))# トレーニングデータ内の各クラスのサンプル数
print("test: ", np.unique(b_test, return_counts=True))    # テストデータ内の各クラスのサンプル数

### [詳細] データの分割

　「分類」でも、モデルの学習とモデルの評価のために、データセットを分割して、トレーニングデータ（学習用データ）とテストデータ（評価用データ）を作成します。

　「回帰」のときとほぼ同じコードを使いますが、一点だけ異なっています。オプションに`stratify`を追加している点です。このオプションは、分割前のデータセットに含まれるラベルの割合を、分割後のトレーニングデータやテストデータでも同じ割合に維持するオプションです。このオプションには、ラベルデータを指定します。

```python
# 書き方例
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0, stratify=y)
```


In [0]:
# (1) データの分割: トレーニングデータ80%, テストデータ20%
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y, random_state=0)

# 確認
print("original: ", np.unique(y, return_counts=True))      # 元のデータセット内の各クラスのサンプル数
print("training: ", np.unique(y_train, return_counts=True))# トレーニングデータ内の各クラスのサンプル数
print("test: ", np.unique(y_test, return_counts=True))    # テストデータ内の各クラスのサンプル数

### [詳細] スケーリング

　分類アルゴリズムのいくつかは、「説明変数」のスケーリングを推奨しています。ここでは、「標準化 Standardization」（平均0、標準偏差1に変換）をおこなっています。

- スケーリングを推奨しているアルゴリズム
  - ロジスティック回帰
  - サポートベクトルマシン
  - ニューラルネットワーク
- スケーリング不要なアルゴリズム
  - 決定木
  - ランダムフォレスト

```python
# 書き方
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train_ss = ss.fit_transform(トレーニングデータの説明変数)
x_test_ss = ss.transform(テストデータの説明変数)
```

<small>*※ スケーリングの有無による違いも確認できるように、スケーリング後のデータには新しい変数名を付けています。*</small>

In [0]:
# (2) スケーリング: 標準化
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train_ss = ss.fit_transform(x_train)
x_test_ss = ss.transform(x_test)

# 確認
print(x_test_ss)

### [詳細] ラベルの数値変換

　「ラベル変数」を次のように整数値「0,1,2,...」に変換します。必須でないこともありますが、推奨されている操作です。
  - `A` → `0`
  - `B` → `1`

<small>*※ あとで使用する分類結果を可視化する関数`draw_decision_boundary`には、数値化ラベルを使うので、ここでは必須です。*</small>

```python
# 書き方
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_le = le.fit_transform(トレーニングデータのラベル変数)
y_test_le  = le.transform(テストデータのラベル変数)
```

In [0]:
# (3) ラベルの数値変換
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# トレーニングデータ
le.fit(y_train)
y_train_le = le.transform(y_train)
#y_train_le = le.fit_transform(y1_train)　# 上の2行と同じ

# テストデータ
y_test_le = le.transform(y_test)

# 確認
print(y_test_le)

# 整数変換されたラベルを元に戻すことも可能
print(le.inverse_transform(y_test_le))

　上述のように、ラベル変数を整数値「0,1,2,...」に変換する方法の他に、**One-hot**表現に変換する方法があります。Irisデータセットを例にすると、`Iris-setosa`の場合、`Iris-setosa=1, Iris-versicolor=0, Iris-virginica=0`のように、一つのスイッチのみが「1」、その他のスイッチは「0」の形に変換する方法です。次のようなリスト状のデータに変換します。
- Irisデータセット
  - `Iris-setosa` → `[1,0,0]`
  - `Iris-versicolor` → `[0,1,0]`
  - `Iris-virginica` → `[0,0,1]`

- 今回のデータセットの場合、
  - `A` → `[1,0]`
  - `B` → `[0,1]`

```python
# 書き方
from sklearn.preprocessing import OneHotEncoder
oe = OneHotEncoder()
y_train_oe = oe.fit_transform(トレーニングデータのラベル変数)
y_test_oe  = oe.transform(テストデータのラベル変数)
```

　今回のscikit-learnのプログラムでは、ラベル変数をOne-hot表現にする必要はありません。しかし、Irisiデータセットのような、多クラス分類ではよく使われる変換方法ですので、こちらもよく使われるので覚えておくと良いでしょう。

In [0]:
# One-hot表現に変換
from sklearn.preprocessing import OneHotEncoder
oe = OneHotEncoder(handle_unknown='ignore')

# トレーニングデータ
y_train_oe = oe.fit_transform(y_train.reshape(-1, 1))

# テストデータ
y_test_oe = oe.transform(y_test.reshape(-1, 1))

# 確認: .toarray()をつけた方がわかりやすい
print(y_test_oe.toarray())

## ロジスティック回帰 Logistic regression

　**ロジスティック回帰（Logisitic regression）**は、線形回帰に少し手を加えて、**あるクラスに分類される確率を求める手法**です。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/logistic_regression.png?raw=true" alt="logistic_regression" height="250px">

　ロジスティック回帰の式は、次のとおりです（詳しくは「ロジスティック回帰式の導出」参照）。

$$ z = \beta_1x_1 + \beta_2x_2 + ... + \beta_kx_k + ... + \beta_mx_m + \epsilon $$
$$ p = \frac{1}{1+e^{-z}} $$

- $k$番目($k=1,2,3,...,m$)の係数$\beta$と説明変数$x$をそれぞれ$\beta_k$、$x_k$で表しています。$\epsilon$（epsilon）は誤差（切片）です。
- $p$はあるクラスに属している確率を表しています。$e$はネイピア数（=2.7182...）です。  

　このロジスティック回帰モデルをトレーニングすると、係数$\beta$や誤差$\epsilon$が求まります。その予測モデルに、新しいデータ$x=(x_1, x_2, ..., x_k, ..., x_m)$を与えると、あるクラスに属している確率$p$がわかります。

　どうやってモデルの学習するかについては、後述の「ロジスティック回帰のトレーニング方法」を参照してください。


### ロジスティック回帰の式を詳しく...
<small>*※ 読み飛ばしてOKです。ロジスティック回帰の式の背景を数式を使って説明しています。*</small>

　ロジスティック回帰の式を理解するためには、**オッズ(odds)**から始める必要があります。

　ある事象Aが起こる確率を$p$としたとき、Aが起こらない確率は$1-p$となります。このとき、Aの起こる確率とAが起こらない確率の比は、$$\frac{p}{1-p}$$となります。これがオッズと呼ばれるもので、Aが起こる「見込み」を表しています。ここで、オッズの意味を深く理解する必要はありません。最終的に知りたいことは、**データから確率$p$を推測する式**です。

　オッズの対数（**対数オッズ**）をとり、関数とみなすと、$$f(p) = \log{\frac{p}{1-p}}$$となります。これは、**ロジット関数（logit）**として知られる関数です。

　ロジット関数を逆関数にすると、$f(p)$から$p$を求める関数が得られます。
\begin{align*}
f(p) &= \log{\frac{p}{1-p}} \\
e^{f(p)} &= \frac{p}{1-p} \\
e^{f(p)} - e^{f(p)} p &= p \\
p &= {\frac{e^{f(p)}}{e^{f(p)}+1}} \\ &= \frac{1}{1+e^{-f(p)}}
\end{align*}

　ここで、$f(p) = z$ とすると、ロジスティック回帰のふたつ目の式が得られます。これは、**ロジスティック関数（logistic）**として知られる関数です。$$ p = \frac{1}{1+e^{-z}} $$この関数は、入力値$z$からAが起こる確率$p$を求める関数とみなせます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/logistic_curve.png?raw=true" alt="logistic_curve" height="200px">

　ロジスティック回帰は、線形回帰の式をロジスティック関数の入力$z$につなげたものです。これは、「データ$x$」を「Aがおこる確率$p$」に変換する式です。

$$ z = \beta_1x_1 + \beta_2x_2 + ... + \beta_kx_k + ... + \beta_mx_m + \epsilon $$

$$ p = \frac{1}{1+e^{-(\beta_1x_1 + \beta_2x_2 + ... + \beta_kx_k + ... + \beta_mx_m + \epsilon)}} $$

　

### ロジスティック回帰モデルのコスト関数
<small>*※ 読み飛ばしてOKです。*</small>

　ロジスティック回帰モデルも、線形回帰と同じく、正解ラベルと予測ラベルが最も一致するように学習をおこないます。異なるのは、ロジスティック回帰の出力が「確率」である点です。そこで、ロジスティック回帰は、正解ラベルと一致している確率が最も高くなるように学習します。それを表したのが次の式です。

$$L = \prod_{i=1}^{n}(p_{i})^{y_{i}}(1-p_{i})^{1-y_{i}}$$

ここで、$i$はサンプル番号、$p_i$はラベル「1」である予測確率、$y_i$は正解ラベルを表しています。この式は、**尤度 Likelihood**と呼ばれる式です。

　仮に、1番目のサンプルの正解ラベルが$y_1=1$であった場合、
$$L_1 = (p_1)^{1}（1-p_1）^{1-1}=p_1$$
になります。これは、ラベル「1」である予測確率です。一方で、2番目のサンプルの正解ラベルが$y_2=0$であった場合、
$$L_2 = (p_2)^{0}（1-p_2）^{1-0}=1-p_2$$
になり、ラベル「0」である予測確率（ラベル「1」ではない予測確率）となります。このような計算をすべてのサンプルにわたっておこない、尤度を求めます。
$$例: L = p_1 (1-p_2) p_3 p_4 (1-p_5) ... (1-p_{n-1}) p_n$$

　ロジスティック回帰モデルでは、目的関数「尤度（予測確率の積）」が最大となる$\beta_{optimum}$や$\epsilon_{optimum}$を探索します。実際には、微分を容易にするために（また、小数点数の総積によるアンダフローを防ぐ目的で）、尤度の式を対数（**対数尤度 Log likelihood**）にします。
$$\log{L} = \sum_{i=1}^{n}{ [ y_{i} \log{p_{i}} + (1-y_{i}) \log{(1-p_{i})} ] }  $$

　さらに、勾配法などで最小値を調べるために、上の式に-1をかけて「コスト関数」にしています。これは、**交差エントロピー誤差関数（Cross-entropy error function）**と呼ばれる関数です。

$$ Cost = -\log{L} = \sum_{i=1}^{n}{ [ -y_{i} \log{p_{i}} - (1-y_{i}) \log{(1-p_{i})} ] }$$




### ロジスティック回帰モデルのトレーニング方法

　ロジスティック回帰の $\beta$ や $\epsilon$ の最適解をどうやって求めるか? ロジスティック回帰では、**交差エントロピー誤差関数（Cross-entropy error function）**と呼ばれる関数を最小化する$\beta$ や $\epsilon$ を探索します（Wikipedia「[Cross-entropy error function and logistic regression](https://en.wikipedia.org/wiki/Cross_entropy#Cross-entropy_error_function_and_logistic_regression)」）。この関数は、非常に複雑で、解析的に（最小二乗法のような計算で）$\beta$ や $\epsilon$ を求められません。そこで、勾配法などのパラメータを繰り返し更新する「反復法 Iterative method」で最適解に近づきます。どのような手法が使われているか知りたい場合は、[scikit-learn ユーザーガイド](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)などを参照してください。


　勾配法を使った最適化については、[Python Machine Learning](https://www.packtpub.com/data/python-machine-learning-third-edition)（原書）/ [Python機械学習プログラミング](https://book.impress.co.jp/books/1117101099)（日本語訳）にも概説されています。

### 多クラス(Multi class)の分類

　ここでは2種類の方法を紹介します。

- 一対他 One-vs-Rest (OvR)  
　各クラスに対して、2クラスの分類をおこないます（例えば、「種Aかそれ以外か」「種Bかそれ以外か」「種Cかそれ以外か」）。そうすると、各クラスである予測確率が得られます。そのうち、最も予測確率が高いクラスを「予測値」として採用します。
  - 各確率の合計が1にならないこともあります
  - Aの確率 $P_{A} = 0.70$、A以外の確率 $P_{A} = 0.30$
  - Bの確率 $P_{B} = 0.20$、B以外の確率 $P_{B} = 0.80$
  - Cの確率 $P_{C} = 0.01$、C以外の確率 $P_{C} = 0.99$
  - 予測ラベル: A

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/OvR.png?raw=true" alt="OvR" height="200px">

- Multinomial  
　各クラスの予測確率を同時に得る方法です。最も確率が高いものを「予測値」として採用します。
  - 各確率の合計は1になります
  - Aの確率 $P_{A} = 0.70$
  - Bの確率 $P_{B} = 0.25$
  - Cの確率 $P_{C} = 0.05$
  - 予測ラベル: A

　どの方法が使えるかについては、分類アルゴリズムや最適化手法で決まります。




### 実装

　ロジスティック回帰を実装してみましょう。

```python
# 書き方
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression(ハイパーパラメータ)
model_lr.fit(トレーニングデータ)
```

以下で使っているハイパーパラメータ（詳しくはscikit-learn [LogisiticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)参照）:
- `solver="sag"`: 最適化アルゴリズムに「Stochastic Average Gradient (SAG)」と呼ばれる確率的勾配法を使用する
- `max_iter=100`: トレーニング回数（反復回数）を100回にする


<small>*※ グラフ描画の関数[`draw_decision_boundary`](https://github.com/CropEvol/lecture/blob/master/textbook_2019/modules/classification.py)は、こちらで用意した自作関数です。*</small>

<small>*※ 予測モデルを評価するために、正解率（Accuracy）を調べるコードも書いています。詳しくは、後述の「モデルの評価」で学びます。*</small>


In [0]:
# 使用するデータ
x_train, y_train = x_train_ss, y_train_le  # トレーニングデータ
x_test, y_test  = x_test_ss, y_test_le   # テストデータ

# モデルを作成
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression(solver="sag", max_iter=100, random_state=0)
# モデルをトレーニング
model_lr.fit(x_train, y_train)

# グラフ
from classification import draw_decision_boundary
draw_decision_boundary(x1_train=x_train[:,0], x2_train=x_train[:,1], 
                       x1_test=x_test[:,0], x2_test=x_test[:,1], 
                       y_train=y_train, y_test=y_test, 
                       labels=le.inverse_transform(y_train), 
                       model=model_lr, xlabel="x1", ylabel="x2")

# 係数b
print("Coefficient=", model_lr.coef_)

# 評価（正解率）
print("training data: ", model_lr.score(x_train, y_train)) # トレーニングデータ
print("test data: ",    model_lr.score(x_test, y_test))  # テストデータ

### 実習2

　`LogisticRegression()`内のオプションを変更して、実行結果がどう変わるか（または、変わらないか）を観察してください。なお、オプションの変更には、[`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)のマニュアルページを参照してください。

変更するオプション:
- 最適化手法`solver="sag"`を変更し、別の最適化手法を使用してください。

In [0]:
# 使用するデータ
x_train, y_train = x_train_ss, y_train_le  # トレーニングデータ
x_test, y_test  = x_test_ss, y_test_le   # テストデータ

from sklearn.linear_model import LogisticRegression
# ============== 編集エリア(start) =============
# モデルの作成
model2 = LogisticRegression(solver="sag", max_iter=100, random_state=0)
# ============== 編集エリア(end) ==============
# モデルの学習
model2.fit(x_train, y_train)

# グラフ
from classification import draw_decision_boundary
draw_decision_boundary(x1_train=x_train[:,0], x2_train=x_train[:,1], 
                       x1_test=x_test[:,0], x2_test=x_test[:,1], 
                       y_train=y_train, y_test=y_test, 
                       labels=le.inverse_transform(y_train), 
                       model=model2, xlabel="x1", ylabel="x2")

# 係数b
print("Coefficient=", model2.coef_)

# 評価（正解率）
print("training data: ", model2.score(x_train, y_train)) # トレーニングデータ
print("test data: ",    model2.score(x_test, y_test))  # テストデータ

#### 解答例


　以下では、`solver="lbfgs"`を設定し、「L-BFGS法」と言われる反復法を使ったロジスティック回帰をおこなっています。

In [0]:
# 使用するデータ
x_train, y_train = x_train_ss, y_train_le  # トレーニングデータ
x_test, y_test  = x_test_ss, y_test_le   # テストデータ

from sklearn.linear_model import LogisticRegression
# ============== 編集エリア(start) =============
# モデルの作成
model2 = LogisticRegression(solver="lbfgs", max_iter=100, random_state=0)
# ============== 編集エリア(end) ==============
# モデルの学習
model2.fit(x_train, y_train)

# グラフ
from classification import draw_decision_boundary
draw_decision_boundary(x1_train=x_train[:,0], x2_train=x_train[:,1], 
                       x1_test=x_test[:,0], x2_test=x_test[:,1], 
                       y_train=y_train, y_test=y_test, 
                       labels=le.inverse_transform(y_train), 
                       model=model2, xlabel="x1", ylabel="x2")

# 係数b
print("Coefficient=", model2.coef_)

# 評価（正解率）
print("training data: ", model2.score(x_train, y_train)) # トレーニングデータ
print("test data: ",    model2.score(x_test, y_test))  # テストデータ

## サポートベクトルマシン Suport vector machine (SVM)

　**サポートベクトルマシン（Suport vector machine: SVM）**は、たいへん人気のある分類手法です。この手法は、**最も近いデータまでの距離（マージン margin）が最大となる境界を作り、データを二分する方法**です。この予測モデルは、線形関数+[符号関数（sign関数）](https://ja.wikipedia.org/wiki/%E7%AC%A6%E5%8F%B7%E9%96%A2%E6%95%B0)です。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/svm.png?raw=true" alt="svm" height="250px">

$$ 予測モデル: f(x) = \beta_1x_1 + \beta_2x_2 + ... + \beta_kx_k + ... + \beta_mx_m + e $$

$$
y = \begin{cases} 
1 & (f(x) \geq 0) \\
-1 & (f(x) < 0)
\end{cases} $$

$$ 境界: \beta_1x_1 + \beta_2x_2 + ... + \beta_kx_k + ... + \beta_mx_m + e = 0 $$

- $k$番目($k=1,2,3,...,m$)の係数$\beta$と説明変数$x$をそれぞれ$\beta_k$、$x_k$で表しています。$e$は誤差（切片）です。
- $y=1$であれば「正クラス」、$y=-1$であれば「正クラスではない（負クラス）」と予測します。

　サポートベクトルマシンは、下図のようなイメージです。トレーニングデータを学習して、マージンを最大化できる境界線（または境界面）を探索します。

### サポートベクトルマシンを詳しく...
<small>*※ 読み飛ばしてOKです。数式を使って、サポートベクトルマシンを詳しく説明しています。*</small>

　まず、2つのクラスを完璧に分割できる境界$\beta_1x_1 + \beta_2x_2 + ... + \beta_mx_m + e = 0$を考えます。ベクトルを使って記述をすると、次のように書けます。

$$
境界: \beta^{T} x + e = 0 \\
\beta=\begin{pmatrix}
\beta_1 \\
\beta_2 \\
\vdots \\
\beta_m
\end{pmatrix}, 
x=\begin{pmatrix}
x_1 \\
x_2 \\
\vdots \\
x_m
\end{pmatrix}
$$

　次に、この境界に最も近い正クラスデータ $x_+$ と負クラスデータ $x_-$ について考えます。このような境界に最も近いデータのことを**サポートベクトル**と言います。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/svm_detail1.png?raw=true" alt="svm_detail1" height="200px">

　さらに、正クラス側のサポートベクトルを通り、境界に平行な直線（または面）を考えます。同様に、負クラス側の直線も考えると、次の2つの式を用意できます。これらは、マージンの両端を表す式です。右辺を1と-1にしていますが、この値はなんでも構いません（$e$の値を調整することで、右辺を1や-1にできます）。

\begin{align*}
\beta^{T} x_+ + e &= 1 \\
\beta^{T} x_- + e &= -1
\end{align*}
すなわち、
$$|\beta^{T} x_- + e| = 1$$

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/svm_detail2.png?raw=true" alt="svm_detail2" height="200px">

　正ラベル側のすべての点について考えると、それらの点は、マージンの外側、すなわち、次の不等式で表されるエリアにあるはずです。
$$ \beta^{T} x_i + e \geq 1 $$
正ラベルの値は「1」で表されます。ラベルの値 $t_i=1$ を上の式にかけた不等式は、
$$ t_i (\beta^{T} x_i + e) \geq 1 $$

負ラベル（$t_i=-1$）側についても同様に、
$$ \beta^{T} x_i + e \leq -1 $$
$$ t_i (\beta^{T} x_i + e) \geq 1 $$

したがって、すべての点は次の式で表せます。この式は、最適な$\beta$や$e$を探索する際の条件のひとつです。
$$ t_i (\beta^{T} x_i + e) \geq 1 $$

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/svm_detail3.png?raw=true" alt="svm_detail3" height="200px">

　今度は、最小化関数（コスト関数）について考えていきます。サポートベクトルと境界の距離$d$は、次の式で表されます（これは高校で習った「垂線の長さ（点と直線の距離）」を求める式と同じです）。
$$ d = \frac{| \beta_1 x_1 + \beta_2 x_2 + \dots + \beta_m x_m + e |}{\sqrt{\beta_1^2 + \beta_2^2 + \dots + \beta_m^2 }} = \frac{ |\beta^{T} x + e | }{ \|\beta\| } $$

この式の分子は、$ |\beta^{T} x + e | = 1$より、
$$ d =  \frac{ 1 }{ \|\beta\| } $$

　したがって、境界間の距離（マージン）は、次のとおりです。
$$ \frac{ 2 }{ \|\beta\| },  \|\beta\| = \sqrt{\beta_1^2 + \beta_2^2 + \dots + \beta_m^2} $$

　サポートベクトルマシンは、このマージンを最大化するように学習していきます。「マージンの最大化」は、「$\|w\|$の最小化」と同じです。また、$\|\beta\| = \sqrt{\beta_1^2 + \beta_2^2 + \dots + \beta_m^2} \geq 0 $ のため、「$\|w\|$の最小化」は「$\|w\|^2$の最小化」と同じです。そこで、サポートベクトルマシンは、この$\|\beta\|^2$を2で割った関数を「コスト関数」として、問題を解いています（2で割っているのは、この関数を微分したときの計算が簡単になるからです）。
$$Cost = \frac{\|\beta\|^2}{2}$$


　以上をまとめると、サポートベクトルマシンでは、
$$ 制約条件: t_i (\beta^{T} x_i + e) \geq 1 $$を満たしつつ、
$$ コスト関数: Cost = \frac{\|\beta\|^2}{2}$$
を最小化させる$\beta$と$e$を求めます。

　この$\beta$と$e$の最適値を求める方法については、さらに複雑です。通常、「逐次最小問題最適化法 Sequential Minimal Optimization (SMO)」と呼ばれる方法で、解を探索します。

　なお、上述したサポートベクトルマシンの説明は、2つのクラスを完全に線形分離できるという仮定で話をしていました。そのマージンのことを **ハードマージン（Hard margin）** と言います。しかし、実際の問題では、多くの場合、線形方程式では完全に分離できません。そこで、ある程度の誤分類を許容した方法も考案されており、よく利用されています。なお、この誤分類を許容したマージンのことを**ソフトマージン（Soft margin）**と言います。

### カーネルサポートベクトルマシン Kernel support vector machine (Kernel SVM)

　サポートベクトルマシンは、線形方程式では決して分離できない問題にも対応可能です。

　どのように対応するか? **カーネルトリック Kernel trick**と言われる方法を使います。この方法は、オリジナルな低次元のデータから、高次元のデータを作り出し、その高次元データで問題を解決しようという方法です。

　例えば、2個の説明変数 $(x_1, x_2)$ で線形分離できない問題に対して、次のようなカーネル化をおこないます（なお、カーネル化方法は他にもあります）。
$$(x_1, x_2) \rightarrow (x_1^2, \sqrt{2} x_1 x_2, x_2^2)$$
そして、この新しくできた$(x_1^2, \sqrt{2} x_1 x_2, x_2^2)$を使って、SVMモデルを構築します。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/kernelSVM.png?raw=true" alt="kernelSVM" height="200px">

### 実装

　サポートベクトルマシンを実装してみましょう。scikit-learnを使うと簡単にコーディングできます。

```python
# 書き方
from sklearn.svm import SVC
model_lr = SVC(ハイパーパラメータ)
model_lr.fit(トレーニングデータ)
```


以下で使っているハイパーパラメータ（詳しくはscikit-learn [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)参照）:
- `kernel="linear"`: 線形SVM



In [0]:
# 使用するデータ
x_train, y_train = x_train_ss, y_train_le  # トレーニングデータ
x_test, y_test  = x_test_ss, y_test_le   # テストデータ

# モデルを作成
from sklearn.svm import SVC
model_svm = SVC(kernel="linear", random_state=0) # 線形SVM
# モデルをトレーニング
model_svm.fit(x_train, y_train)

# グラフ
from classification import draw_decision_boundary
draw_decision_boundary(x1_train=x_train[:,0], x2_train=x_train[:,1], 
                       x1_test=x_test[:,0], x2_test=x_test[:,1], 
                       y_train=y_train, y_test=y_test, 
                       labels=le.inverse_transform(y_train), 
                       model=model_svm, xlabel="x1", ylabel="x2")

# 係数b
if model_svm.kernel == "linear": 
  print("Coefficient=", model_svm.coef_)

# 評価（正解率）
print("training data: ", model_svm.score(x_train, y_train)) # トレーニングデータ
print("test data: ",    model_svm.score(x_test, y_test))  # テストデータ

### 実習3

　`SVC()`内のオプションを変更して、その実行結果を観察してください。なお、オプションの変更には、[`SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)のマニュアルページを参照してください。

変更するオプション:  
- カーネル化オプション`kernel="linear"`（線形SVM）を変更し、他のカーネル化手法を使用してください。







In [0]:
# 使用するデータ
x_train, y_train = x_train_ss, y_train_le  # トレーニングデータ
x_test, y_test  = x_test_ss, y_test_le   # テストデータ

from sklearn.svm import SVC
# ============== 編集エリア(start) =============
# モデルの作成
model2 = SVC(kernel="linear", random_state=0)
# ============== 編集エリア(end) ==============
# モデルの学習
model2.fit(x_train, y_train)

# グラフ
from classification import draw_decision_boundary
draw_decision_boundary(x1_train=x_train[:,0], x2_train=x_train[:,1], 
                       x1_test=x_test[:,0], x2_test=x_test[:,1], 
                       y_train=y_train, y_test=y_test, 
                       labels=le.inverse_transform(y_train), 
                       model=model2, xlabel="x1", ylabel="x2")

# 係数b
if model2.kernel == "linear": 
  print("Coefficient=", model2.coef_)

# 評価（正解率）
print("training data: ", model2.score(x_train, y_train)) # トレーニングデータ
print("test data: ",    model2.score(x_test, y_test))  # テストデータ

#### 解答例

　以下のコードでは、カーネル化の手法に「RBF」を使っています。

In [0]:
# 使用するデータ
x_train, y_train = x_train_ss, y_train_le  # トレーニングデータ
x_test, y_test  = x_test_ss, y_test_le   # テストデータ

from sklearn.svm import SVC
# ============== 編集エリア(start) =============
# モデルの作成
model2 = SVC(kernel="rbf", random_state=0)
# ============== 編集エリア(end) ==============
# モデルの学習
model2.fit(x_train, y_train)

# グラフ
from classification import draw_decision_boundary
draw_decision_boundary(x1_train=x_train[:,0], x2_train=x_train[:,1], 
                       x1_test=x_test[:,0], x2_test=x_test[:,1], 
                       y_train=y_train, y_test=y_test, 
                       labels=le.inverse_transform(y_train), 
                       model=model2, xlabel="x1", ylabel="x2")

# 係数b
if model2.kernel == "linear": 
  print("Coefficient=", model2.coef_)

# 評価（正解率）
print("training data: ", model2.score(x_train, y_train)) # トレーニングデータ
print("test data: ",    model2.score(x_test, y_test))  # テストデータ

## 決定木 Decision tree

　**決定木（Decision tree）**は、**説明変数に対する問い（True or False）を繰り返して、木構造の分類モデルを作る方法**です。出来上がった予測モデルは、他の機械学習のモデルに比べると、理解しやすいです。しかし、決定木の予測精度はそれほど高くありません。

　ここで決定木の勉強をするのは、次に紹介する「ランダムフォレスト」のためです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/decision_tree.png?raw=true" alt="decision_tree" height="350px">


### 各分岐をどのように設定するか？



　決定木の各分岐は、適当に設定されているわけではありません。"なるべく少ない質問でクラス分けする"ように木構造を作っていきます（木に例えると、枝を伸ばしていきます）。

　どの説明変数を分岐の質問に使うか？　分割前と分割後の「**情報量**または**不純度**」の差（**情報利得 Information gain**）が大きくなる質問を、あらゆる質問の中から選択します。「情報量」や「不純度」は、多様なラベル（クラス）がそのデータセットにどのぐらい含まれているかを表す指標です。「情報利得」は、分岐によりどれだけクラス分けできるかを表す指標です。

$$情報利得 = I_{pre} - (I_{true} + I_{false})$$

- $I_{pre}$は分割前データの情報量（不純度）
- $I_{true}$は分割後Trueデータの情報量（不純度）
- $I_{false}$は分割後Falseデータの情報量（不純度）

　決定木では、情報量を表す指標として、**エントロピー（Entropy）**が使われます。また、不純度を表す指標には、**ジニ不純度（Gini impurity）**と呼ばれる指標が使われます。


　



### 分岐の設定方法を直感的に理解する

　次のようなデータセットから、鳥類・哺乳類の分類モデル（決定木）を作りたいとします。最初の質問にはどの説明変数を使うと良いでしょうか？

||A. 食性|B. 発生形態|C. 体温|分類ラベル（マーク）|
|:---|---:|---:|---:|---:|
|ニワトリ|草食|卵生|恒温|鳥類（●）|
|ペンギン|肉食|卵生|恒温|鳥類（●）|
|カモノハシ|肉食|卵生|恒温|哺乳類（○）|
|ウシ|草食|胎生|恒温|哺乳類（○）|
|ヒツジ|草食|胎生|恒温|哺乳類（○）|
|ライオン|肉食|胎生|恒温|哺乳類（○）|

　それぞれの説明変数で分けてみると、次のようになります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/decision_tree_which_features.png?raw=true" alt="decision_tree_which_features" height="230px">

- 「A. 食性」で分けた場合、分割後のそれぞれのサブセットの中には、まだ両方のラベルが混ざっています。ラベルの比率は、どちらのサブセットも元のデータセットの比率が維持されています。この質問は、ラベル分割に関して言えば、ほとんど情報を得ることができない質問です。
- 「B. 発生形態」で分けた場合、「胎生」サブセットは、両方のラベルが含まれています。ラベル比率は、元のデータセットからすると、鳥類（●）の割合が高くなっています。一方、「卵生」サブセット側は、1種類のラベルのみとなっています。この質問は、"元のデータセットから哺乳類（○）の多くを分ける"質問であることがわかります。すなわち、ラベル分割に関して、情報を得ることが可能な質問です。
- 「C. 体温」を使った場合、全く分割できません。この説明変数を使ったとしても、元のデータセットと同じ状態が維持されており、何も情報を得ることができていません。



　"なるべく少ない質問でクラス分け" するためには、最も情報が得られそうな「B. 発生形態」を最初の質問に採用するのが良さそうです。



### エントロピー 、ジニ不純度、情報利得
<small>*※ 読み飛ばしてOKです。「各分岐をどのように設定するか？」や「分岐の設定方法を直感的に理解する」を数式を使って、詳しく説明しています。*</small>

#### 情報利得

　決定木の学習アルゴリズムにおける目的関数は、「情報利得（Information gain）」です。情報利得を最大化するように、分岐を作成し、木構造を発達させていきます。情報利得を定式化すると次のようになります。

$$ 情報利得: IG = I(D_{pre}) - (\frac{N_{true}}{N_{pre}}I(D_{true}) + \frac{N_{false}}{N_{pre}}I(D_{false})) $$ 

- $I(D_{pre})$は、データセットにおけるエントロピーまたはジニ不純度
- $I(D_{true})$は、分割後のTrueデータセットにおけるエントロピーまたはジニ不純度
- $I(D_{false})$は、分割後のFalseデータセットにおけるエントロピーまたはジニ不純度
- $N_{pre}$は、データセットのサンプル数
- $N_{false}$は、分割後のTrueデータセットのサンプル数
- $N_{false}$は、分割後のFalseデータセットのサンプル数

　次に、$I(D)$についてです。これにはいくつか種類あり、「エントロピー（Entropy）」または「ジニ不純度（Gini impurity）」がよく使われます。どちらの指標も、データセット中に含まれる各クラスの割合をその計算に使います。以下では、ラベルAとラベルBの分類を例にしており、データセット中のそれぞれのラベルの割合を$p_{A}$、$p_{B}$と表記しています。

#### エントロピー
　データセットの「エントロピー」は、次の式で計算されます。なお、詳細は省きますが、この式は、データセットの平均情報量（情報量の期待値）を求める式でもあります。

$$エントロピー: I(D) = p_{A}log_2 \frac{1}{p_{A}} + p_{B}log_2 \frac{1}{p_{B}} = -p_{A}log_2 p_{A} - p_{B}log_2 p_{B}$$

#### ジニ不純度
　　データセットの「ジニ不純度」は、次の式で求めます。

$$ジニ不純度: I(D) = p_{A}(1 - p_{A}) + p_{B}(1 - p_{B})$$

#### 計算例
　上述「分岐の設定方法を直感的に理解する」の例において、それぞれの説明変数の「エントロピーを使った情報利得」や「ジニ不純度を使った情報利得」がいくつになるか、以下で求めてみます。

||A. 食性|B. 発生形態|C. 体温|分類ラベル（マーク）|
|:---|---:|---:|---:|---:|
|ニワトリ|草食|卵生|恒温|鳥類（●）|
|ペンギン|肉食|卵生|恒温|鳥類（●）|
|カモノハシ|肉食|卵生|恒温|哺乳類（○）|
|ウシ|草食|胎生|恒温|哺乳類（○）|
|ヒツジ|草食|胎生|恒温|哺乳類（○）|
|ライオン|肉食|胎生|恒温|哺乳類（○）|

- エントロピーを使った情報利得
  - A. 食性
$$
I(D_{pre}) = - \frac{2}{6} log_{2} \frac{2}{6} - \frac{4}{6} log_{2} \frac{4}{6} = 0.9182... \\
I(D_{true}) = -\frac{1}{3} log_{2} \frac{1}{3} - \frac{2}{3} log_{2} \frac{2}{3} = 0.9182... \\
I(D_{false}) = - \frac{1}{3} log_{2} \frac{1}{3} - \frac{2}{3} log_{2} \frac{2}{3} = 0.9182... \\
IG = I(D_{pre}) - ( \frac{3}{6} I(D_{true}) + \frac{3}{6} I(D_{false}) ) = 0
$$
  - B. 発生形態
$$
I(D_{pre}) = 0.9182... \\
I(D_{true}) = 0.9182... \\
I(D_{false}) = 0 \\
IG = I(D_{pre}) - ( \frac{3}{6} I(D_{true}) + \frac{3}{6} I(D_{false}) ) = 0.4591...
$$
  - C. 体温
$$
I(D_{pre}) = 0.9182... \\
I(D_{true}) = 0.9182... \\
IG = I(D_{pre}) - \frac{6}{6} I(D_{true}) = 0
$$
  - 分岐の質問には『「B. 発生形態」は卵生/胎生？』が選ばれます。

- ジニ不純度を使った情報利得
  - A. 食性
$$
I(D_{pre}) = \frac{2}{6} (1-\frac{4}{6}) + \frac{4}{6} (1-\frac{2}{6}) = 0.4444... \\
I(D_{true}) = \frac{1}{3} (1-\frac{1}{3}) + \frac{2}{3} (1-\frac{2}{3}) = 0.4444... \\
I(D_{false}) = \frac{1}{3} (1-\frac{1}{3}) + \frac{2}{3} (1-\frac{2}{3}) = 0.4444... \\
IG = I(D_{pre}) - ( \frac{3}{6} I(D_{true}) + \frac{3}{6} I(D_{false}) ) = 0
$$
  - B. 発生形態
$$
I(D_{pre}) = 0.4444... \\
I(D_{true}) = 0.4444... \\
I(D_{false}) = 0 \\
IG = I(D_{pre}) - ( \frac{3}{6} I(D_{true}) + \frac{3}{6} I(D_{false}) ) = 0.2222...
$$
  - C. 体温
$$
I(D_{pre}) = 0.4444... \\
I(D_{true}) = 0.4444... \\
IG = I(D_{pre}) - \frac{6}{6} I(D_{true}) = 0
$$
  - 分岐の質問には『「B. 発生形態」は卵生/胎生？』が選ばれます。
　
- どちらの評価指標を使っても、通常はほとんど同じ結果になります。

### 決定木では、説明変数をスケーリングしなくて良い

　各データの順序は、スケーリングの前後で変わりません。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/scalling.png?raw=true" alt="scalling" height="300px">

　決定木による解析において、スケーリングする前とスケーリングした後では、閾値の値が変更されるのみで、スケーリングの有無で分岐の結果が変わることはありません。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/decision_tree_with_scalling.png?raw=true" alt="decision_tree_with_scalling" height="250px">

### 実装

　scikit-learnを使って、決定木を実装してみましょう。

```python
# 書き方
from sklearn.tree import DecisionTreeClassifier
model_dt = DecisionTreeClassifier(ハイパーパラメータ)
model_dt.fit(トレーニングデータ)
```


以下で使っているハイパーパラメータ（詳しくはscikit-learn [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)参照）:
- `criterion="gini"`: 不純度の指標として「ジニ不純度」を使う
- `max_depth=None`: 決定木の深さの最大値を「上限なし」に設定する


In [0]:
# 使用するデータ
# （説明変数は、スケーリング前のデータ）
x_train, y_train = x_train_raw, y_train_le  # トレーニングデータ
x_test, y_test  = x_test_raw, y_test_le   # テストデータ

# モデルを作成
from sklearn.tree import DecisionTreeClassifier
model_dt = DecisionTreeClassifier(criterion="gini", max_depth=None, random_state=0)

# モデルをトレーニング
model_dt.fit(x_train, y_train)

# グラフ
from classification import draw_decision_boundary
draw_decision_boundary(x1_train=x_train[:,0], x2_train=x_train[:,1], 
                       x1_test=x_test[:,0], x2_test=x_test[:,1], 
                       y_train=y_train, y_test=y_test, 
                       labels=le.inverse_transform(y_train), 
                       model=model_dt, xlabel="x1", ylabel="x2")

# 評価（正解率）
print("training data: ", model_dt.score(x_train, y_train)) # トレーニングデータ
print("test data: ",    model_dt.score(x_test, y_test))  # テストデータ

　決定木を可視化すると、

In [0]:
# === メジャーな方法 ===
import graphviz
from sklearn.tree import export_graphviz
# 予測モデルをグラフ用データに変換
export_graphviz(model_dt, out_file="tree.dot", 
                feature_names=cols, 
                class_names=["A", "B"], 
                filled=True, rounded=True)
# 表示
with open('tree.dot') as g:
  dot_graph = g.read()
graphviz.Source(dot_graph)

In [0]:
# === マイナーな方法（おすすめ） ===
from dtreeviz.trees import dtreeviz

# 予測モデルをグラフ用データに変換
viz = dtreeviz(model_dt, x_train, y_train, 
    feature_names=cols, 
    class_names=["A", "B"], 
    target_name='species',
    orientation ='LR')
# 表示
viz

### 過学習（オーバーフィッティング Overfitting）

　トレーニングデータの正解率が高い一方で、テストデータの正解率が低い状態を、**過学習（オーバーフィッティング overfitting）**と言います。トレーニングデータに"合わせ過ぎた"予測モデルが作られており、新しいデータに対する予測精度を損なっている状態です。

　過学習に陥るおもな要因は2つです。
- トレーニングデータの量が少ない。もしくは、偏ったデータのみをトレーニングデータとして使っている。
- モデルが複雑すぎる。

　ひとつ目のトレーニングデータ量や質の問題は、データの量を増やすか、質を改善するかしかありません。

　ふたつ目のモデルの複雑さについては、より単純なモデルを選択することで解決できる場合があります。

　決定木の場合、木の深さを設定しない場合（すなわち、複雑なモデルの場合）、すべてのサンプルを強引にクラス分けするため、過学習になりやすいです。木の深さを制限して、単純なモデルを構築するか、別の手法（ランダムフォレストなど）を用いることで解決できるかもしれません。

### 実習4

　`DecisionTreeClassifier()`内のオプションを変更して、その実行結果を観察してください。なお、オプションの変更には、[`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)のマニュアルページを参照してください。

変更するオプション:  
- 決定木の深さ`max_depth=None`（上限なし）を、`max_depth=2`に変更してください。
- 不純度の指標`criterion="gini"`（ジニ不純度）を変更し、「エントロピー」を使用するようにしてください。


　また、使用する説明変数を、スケーリングありに変更しても、同じ結果が得られることを確認してください。

In [0]:
# 使用するデータ
# スケーリング前のデータ
x_train, y_train = x_train_raw, y_train_le     # トレーニングデータ
x_test, y_test  = x_test_raw, y_test_le      # テストデータ
# スケーリング後のデータ
#x_train, y_train = x_train_ss, y_train_le  # トレーニングデータ
#x_test, y_test  = x_test_ss, y_test_le   # テストデータ

from sklearn.tree import DecisionTreeClassifier
# ============== 編集エリア(start) =============
# モデルの作成
model2 = DecisionTreeClassifier(criterion="gini", max_depth=None, random_state=0)
# ============== 編集エリア(end) ==============
# モデルの学習
model2.fit(x_train, y_train)

# グラフ
from classification import draw_decision_boundary
draw_decision_boundary(x1_train=x_train[:,0], x2_train=x_train[:,1], 
                       x1_test=x_test[:,0], x2_test=x_test[:,1], 
                       y_train=y_train, y_test=y_test, 
                       labels=le.inverse_transform(y_train), 
                       model=model2, xlabel="x1", ylabel="x2")

# 評価（正解率）
print("training data: ", model2.score(x_train, y_train)) # トレーニングデータ
print("test data: ",    model2.score(x_test, y_test))  # テストデータ

# 予測モデルをグラフ用データに変換
from dtreeviz.trees import dtreeviz
viz = dtreeviz(model2, x_train, y_train, 
    feature_names=cols, 
    class_names=["A", "B"], 
    target_name='species',
    orientation ='LR')
# 表示
viz

#### 解答例

　以下では、`criterion="entropy"`と`max_depth=2`を同時に設定しています。

In [0]:
# 使用するデータ
# スケーリング前のデータ
x_train, y_train = x_train_raw, y_train_le     # トレーニングデータ
x_test, y_test  = x_test_raw, y_test_le      # テストデータ
# スケーリング後のデータ
#x_train, y_train = x_train_ss, y_train_le  # トレーニングデータ
#x_test, y_test  = x_test_ss, y_test_le   # テストデータ

from sklearn.tree import DecisionTreeClassifier
# ============== 編集エリア(start) =============
# モデルの作成
#model2 = DecisionTreeClassifier(criterion="gini", max_depth=2, random_state=0)
#model2 = DecisionTreeClassifier(criterion="entropy", max_depth=None, random_state=0)
model2 = DecisionTreeClassifier(criterion="entropy", max_depth=2, random_state=0)

# ============== 編集エリア(end) ==============
# モデルの学習
model2.fit(x_train, y_train)

# グラフ
from classification import draw_decision_boundary
draw_decision_boundary(x1_train=x_train[:,0], x2_train=x_train[:,1], 
                       x1_test=x_test[:,0], x2_test=x_test[:,1], 
                       y_train=y_train, y_test=y_test, 
                       labels=le.inverse_transform(y_train), 
                       model=model2, xlabel="x1", ylabel="x2")

# 評価（正解率）
print("training data: ", model2.score(x_train, y_train)) # トレーニングデータ
print("test data: ",    model2.score(x_test, y_test))  # テストデータ

from dtreeviz.trees import dtreeviz

# 予測モデルをグラフ用データに変換
viz = dtreeviz(model2, x_train, y_train, 
    feature_names=cols, 
    class_names=["A", "B"], 
    target_name='species',
    orientation ='LR')
# 表示
viz

## ランダムフォレスト Random forest

　**ランダムフォレスト（Random forest）**は、複数の決定木を作り、**各決定木の予測結果を多数決して、最終的な予測値を決める手法**です。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/random_forest.png?raw=true" alt="random_forest" height="250px">

### アルゴリズム

ランダムフォレストは、複数の決定木を使ったモデルです。このモデルのアルゴリズムは次のとおりです。
1. オリジナルのデータセットから、$n$個のサンプルをランダムに抽出します。この時、復元抽出（重複を許して抽出）をおこないます。このようなサンプリング方法を、**ブートストラップサンプリング（Bootstrap sampling）**といいます。

1. このブートストラップ標本を使って、決定木を構築します。

1. 1,2のステップを$k$回繰り返します。得られたk個の決定木がランダムフォレストです。

1. 新しいデータのラベルを予測する際には、各決定木から予測値を集め、多数決により最終的な予測値を決定します。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/random_forest_algorithm.png?raw=true" alt="random_forest_algorithm" height="180px">


### 実装

　ランダムフォレストを実装してみましょう。

```python
# 書き方
from sklearn.ensemble import RandomForestClassifier
model_dt = RandomForestClassifier(ハイパーパラメータ)
model_dt.fit(トレーニングデータ)
```

以下で使っているハイパーパラメータ（詳しくはscikit-learn [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)参照）:
- `n_estimators=100`: 決定木の数
- `criterion="gini"`: 不純度の指標として「ジニ不純度」を使う 
- `max_depth=2`: 木の深さの最大値を「2」に設定する




In [0]:
# 使用するデータ
# （説明変数は、スケーリング前のデータ）
x_train, y_train = x_train_raw, y_train_le  # トレーニングデータ
x_test, y_test  = x_test_raw, y_test_le   # テストデータ

# モデルを作成
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=100, criterion="gini", max_depth=2, random_state=0)

# モデルをトレーニング
model_rf.fit(x_train, y_train)

# グラフ
from classification import draw_decision_boundary
draw_decision_boundary(x1_train=x_train[:,0], x2_train=x_train[:,1], 
                       x1_test=x_test[:,0], x2_test=x_test[:,1], 
                       y_train=y_train, y_test=y_test, 
                       labels=le.inverse_transform(y_train), 
                       model=model_rf, xlabel="x1", ylabel="x2")

# 各説明変数の重要度
print('Importances:', model_rf.feature_importances_ )

# 評価（正解率）
print("training data: ", model_rf.score(x_train, y_train)) # トレーニングデータ
print("test data: ",    model_rf.score(x_test, y_test))  # テストデータ

### 各説明変数の重要度

　ランダムフォレストでは、`モデル名.feature_importances_`により、**各説明変数の重要度（Feature importances）**、すなわち、クラス分けに重要な説明変数を調べることが可能です。この重要度はどのようにして算出されているか？ **Permutation Importance**と呼ばれる手法を使って、各説明変数の重要度を調べています。

　「Permutation Importance」の概要は次のとおりです。

1. 説明変数をひとつ選びます。

1. 選んだ説明変数のデータをシャッフルします。
1. その結果、精度がどの程度下がるかを調べます。
  - ほとんど精度が変わらないようであれば、その説明変数の重要度は低い
  - 精度が劇的に下がるようであれば、その重要度は高い
1. 1-3をすべての説明変数についておこないます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/permutation_importance.png?raw=true" alt="permutation_importance" height="300px">



### 実習5

　`RandomForestClassifier()`内のオプションを変更して、その実行結果を観察してください。なお、オプションの変更には、[`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)のマニュアルページを参照してください。

変更するオプション:  
- ブートストラップサンプリングの数（決定木の数）を`n_estimators=10`に変更してください。
- ブートストラップサンプリングの数（決定木の数）を`n_estimators=1000`に変更してください。

　また、あるオプションを追加して、ブートストラッピング（Bootstrapping）、すなわち、復元抽出（重複を許して抽出）をOFFにすると、「決定木」と同じ結果が得られます。マニュアルページで追加するオプションを調べて、それを確かめてください。


In [0]:
# 使用するデータ
# スケーリング前のデータ
x_train, y_train = x_train_raw, y_train_le     # トレーニングデータ
x_test, y_test  = x_test_raw, y_test_le      # テストデータ

from sklearn.ensemble import RandomForestClassifier
# ============== 編集エリア(start) =============
# モデルの作成
model2 = RandomForestClassifier(n_estimators=100, criterion="gini", max_depth=2, random_state=0)
# ============== 編集エリア(end) ==============
# モデルの学習
model2.fit(x_train, y_train)

# グラフ
from classification import draw_decision_boundary
draw_decision_boundary(x1_train=x_train[:,0], x2_train=x_train[:,1], 
                       x1_test=x_test[:,0], x2_test=x_test[:,1], 
                       y_train=y_train, y_test=y_test, 
                       labels=le.inverse_transform(y_train), 
                       model=model2, xlabel="x1", ylabel="x2")

# 各説明変数の重要度
print('Importances:', model2.feature_importances_ )

# 評価（正解率）
print("training data: ", model2.score(x_train, y_train)) # トレーニングデータ
print("test data: ",    model2.score(x_test, y_test))  # テストデータ

#### 解答例

　以下のコードでは、`n_estimators=10`に設定しています。

In [0]:
# 使用するデータ
# スケーリング前のデータ
x_train, y_train = x_train_raw, y_train_le     # トレーニングデータ
x_test, y_test  = x_test_raw, y_test_le      # テストデータ
# スケーリング後のデータ
#x_train, y_train = x_train_ss, y_train_le  # トレーニングデータ
#x_test, y_test  = x_test_ss, y_test_le   # テストデータ

from sklearn.ensemble import RandomForestClassifier
# ============== 編集エリア(start) =============
# モデルの作成
model2 = RandomForestClassifier(n_estimators=10, criterion="gini", max_depth=2, random_state=0)
#model2 = RandomForestClassifier(n_estimators=100, criterion="gini", max_depth=2, random_state=0)
#model2 = RandomForestClassifier(n_estimators=1000, criterion="gini", max_depth=2, random_state=0)
#model2 = RandomForestClassifier(n_estimators=100, criterion="gini", max_depth=2, random_state=0, bootstrap=False)

# ============== 編集エリア(end) ==============
# モデルの学習
model2.fit(x_train, y_train)

# グラフ
from classification import draw_decision_boundary
draw_decision_boundary(x1_train=x_train[:,0], x2_train=x_train[:,1], 
                       x1_test=x_test[:,0], x2_test=x_test[:,1], 
                       y_train=y_train, y_test=y_test, 
                       labels=le.inverse_transform(y_train), 
                       model=model2, xlabel="x1", ylabel="x2")

# 各説明変数の重要度
print('Importances:', model2.feature_importances_ )

# 評価（正解率）
print("training data: ", model2.score(x_train, y_train)) # トレーニングデータ
print("test data: ",    model2.score(x_test, y_test))  # テストデータ

## ニューラルネットワーク Neural network

　**ニューラルネットワーク（Neural network）**は、身の回りにある多くのIoT機器（インターネットに接続された電子機器）やWebサービスの内部でよく使われている機械学習手法です（例をあげると、Google翻訳やデジタルカメラやスマホカメラの顔認識機能、など）。

　**神経細胞が信号を伝達する仕組みを模した機械学習アルゴリズム**が使われています。この予測モデルからは、各分類クラスに含まれる確率が出力されます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/neural_network.png?raw=true" alt="neural_network" height="250px">

　なお、ニューラルネットワークには多くの種類があります（参考: [The Neural Network Zoo](https://www.asimovinstitute.org/neural-network-zoo/) / THE ASIMOV INSTITUTE）。


### ニューラルネットワークのユニット

　ニューラルネットワークは、入力値（Input）である数値データが、入力層、中間層、出力層を経て、出力値（Output）に変換される機械学習手法です。各層は、1個以上の**ユニット**（上述の図では、$x_1$、$h_{11}$、$y$など、円で描かれているもの）で構成されています。

　以下では、各層のユニットがどのような値を受け取るかを、どのような値を出力するかを説明しています。

<small>*※ 各層には、バイアス（誤差）を表すユニットもありますが、ここではバイアスユニットを省略しています。*</small>



#### ◆ 入力層（Input layer）
　この層のユニットの数は、説明変数の数と同じです。各ユニットは、説明変数の数値をそのまま受け取ります。入力層の各ユニットが受け取った数値は、次の層（隠れ層 第1層）の入力データになります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/NN_input_layer.png?raw=true" alt="NN_input_layer" height="300px">



#### ◆ 隠れ層（Hidden layers）
　隠れ層 第1層の各ユニットは、入力層のすべてのユニットからデータを受け取ります。その際、入力層の各ユニットから出力される数値にいくらかの**重み（weight）**をかけた値を受け取ります。さらに、受け取った数値の合計値を **活性化関数（Activation function）** と言われる関数で変換して、次の層への出力値を発生させます。

\begin{align}
例)　t_{11} &= w_{x_{1}\to h_{11}} x_{1} + w_{x_{2}\to h_{11}} x_{2} \\
h_{11} &= activation(t_{11})
\end{align}

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/NN_hidden_layer.png?raw=true" alt="NN_hidden_layer" height="300px">

よく使われる活性化関数:
- ReLU関数
- シグモイド関数（ロジスティック関数）
- tanh関数

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/NN_activation_function.png?raw=true" alt="NN_activation_function" height="200px">

　「活性化関数」は、非線形（直線ではない）関数を基本的には使います。この理由については、下記の「活性化関数に非線形関数を使うのはなぜか？」を参照してください。

　隠れ層　第2層以降についても基本的には同じです。前層のすべてのユニットから出力される数値に重みをかけた値を受け取ります。そして、活性化関数によって、数値の変換をおこなった後、次の層に向けて出力します。

\begin{align}
例)　t_{21} &= w_{h_{11}\to h_{21}} h_{11} + w_{h_{12}\to h_{21}} h_{12} + w_{h_{13}\to h_{21}} h_{13} \\
h_{21} &= activation(t_{21})
\end{align}

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/NN_hidden_layer2.png?raw=true" alt="NN_hidden_layer2" height="300px">

#### ◆ 出力層（output layer）
　
　出力層も、数値の受け取りに関しては、隠れ層と同じです: 隠れ層の最後の層にある各ユニットから出力される数値に重みをかけた値を受け取ります。受け取った数値の合計値を**ソフトマックス関数（Softmax関数）**で変換します。ソフトマックス関数についての詳細は省きますが、変換後の値を各クラスに所属する「確率」とみなして予測値を得ます。

\begin{align}
例)　t_{y} &= w_{h_{31}\to y} h_{31} + w_{h_{32}\to y} h_{32} + w_{h_{33}\to y} h_{33} \\
y &= softmax(t_{y})
\end{align}

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/NN_output_layer.png?raw=true" alt="NN_output_layer" height="300px">

### ニューラルネットワークの重みの更新

　各ユニット間の重みは、モデルの学習で随時更新されます。非線形な活性化関数が間に入っているため、解析的に（計算でピンポイントに）それぞれの重み$w$の最適値を求めることはできません。そこで、確率的勾配降下法などの反復法を使って、重みを徐々に更新し、最適値を探します。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/NN_update_w.png?raw=true" alt="NN_update_w" height="450px">

### 活性化関数に非線形関数を使うのはなぜか？
<small>*※ 読み飛ばしてOKです。*</small>

　この答えは、活性化関数に線形関数（直線になる関数）を使うと、わざわざ隠れ層を作る意味がなくなるからです。

　簡単な例をあげると、次のような3層のネットワークがあったとします（各層のユニット数は1個）。
- 第1層: $t_{1} = a_{1}x$
- 第1層の活性化関数: $y_{1} = b_{1}t_{1} $
- 第2層: $t_{2} = a_{2}y_{1}$
- 第2層の活性化関数: $y_{2} = b_{2}t_{2} $
- 第3層: $t_{3} = a_{3}y_{2}$
- 第3層の活性化関数: $y_{3} = b_{3}t_{3} $

　この6つの式を1行で書くと、
$$y_3 = b_{3}(a_{3}(b_{2}(a_{2}(b_{1}(a_{1}x))))) = a_{1}a_{2}a_{3}b_{1}b_{2}b_{3}x$$

$c = a_{1}a_{2}a_{3}b_{1}b_{2}b_{3}$とすると、

$$ y_3 = c x $$

となり、単層の式で表すことが可能になります。各層のユニット数が増えても同様です。

　活性化関数に線形関数を使った場合、多層のニューラルネットワークを構築しても計算量が増えるだけで、まったくメリットはありません。そのため、活性化関数には非線形関数を使います。





### 実装

　scikit-learnを使ったニューラルネットワークの実装方法は次のとおりです。

```python
# 書き方
from sklearn.neural_network import MLPClassifier
model_nn = MLPClassifier(ハイパーパラメータ)
model_nn.fit(トレーニングデータ)
```

以下で使っているハイパーパラメータ（詳しくはscikit-learn [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)参照）:

- `hidden_layer_sizes=(3,3,3,)`: （上図のように）3層各3モジュールを「隠れ層」として設置する
- `solver="sgd"`: 重みの更新方法に「確率的勾配降下法」を使用する
- `learning_rate_init=0.1`: 重みの更新率を0.1に設定する
- `max_iter=1000`: 学習回数を1000回に設定する

<small>*※ ニューラルネットワーク用のライブラリは、他にも多数あります（TensorFlow、Keras、Pytorch、Caffeなど）。より高度なニューラルネットワークを構築するには、それらのライブラリを使う方が良いでしょう。*</small>


In [0]:
# 使用するデータ
x_train, y_train = x_train_ss, y_train_le  # トレーニングデータ
x_test, y_test  = x_test_ss, y_test_le   # テストデータ

# モデルを作成
from sklearn.neural_network import MLPClassifier
model_nn = MLPClassifier(hidden_layer_sizes=(3,3,3,), solver="sgd", 
                        learning_rate_init=0.1, max_iter=1000, random_state=0)
# モデルをトレーニング
model_nn.fit(x_train, y_train)

# グラフ
from classification import draw_decision_boundary
draw_decision_boundary(x1_train=x_train[:,0], x2_train=x_train[:,1], 
                       x1_test=x_test[:,0], x2_test=x_test[:,1], 
                       y_train=y_train, y_test=y_test, 
                       labels=le.inverse_transform(y_train), 
                       model=model_nn, xlabel="x1", ylabel="x2")

# モジュール間の各コネクションの重み
print("Coefficients=", model_nn.coefs_)

# 評価（正解率）
print("training data: ", model_nn.score(x_train, y_train)) # トレーニングデータ
print("test data: ",    model_nn.score(x_test, y_test))  # テストデータ

## モデルの評価

　二値分類（A / not A）の場合、実測値（実ラベル）とモデルから得られた予測値（予測ラベル）の一致/不一致を調べると、次の4つのタイプの結果が得られます。

||予測ラベル=A|予測ラベル=not A|
|:---|:---|:---|
|**実ラベル=A**|真陽性<br>True positive: TP|偽陰性<br>False negative: FN|
|**実ラベル=not A**|偽陽性<br>False positive: FP|真陰性<br>True negative: TN|

　よく使われる評価指標は、実ラベルと予測ラベルがどれだけ一致していたかを表す指標、**正解率（Accuracy）**です。

$$Accuracy = \frac{TP+TN}{TP + FN + FP + TN}$$

　今回の実習では、各モデルの精度を正解率で評価していきました。再度、それぞれの正解率を確認しておきます。

In [0]:
# ロジスティック回帰モデル
print("=== Logistica regression ===")
print("training data: ", model_lr.score(x_train_ss, y_train_le)) # トレーニングデータ
print("test data: ",    model_lr.score(x_test_ss, y_test_le))  # テストデータ

# サポートベクトルマシンモデル
print("=== Support vector machine ===")
print("training data: ", model_svm.score(x_train_ss, y_train_le)) # トレーニングデータ
print("test data: ",    model_svm.score(x_test_ss, y_test_le))  # テストデータ

# 決定木モデル
print("=== Decision tree ===")
print("training data: ", model_dt.score(x_train_raw, y_train_le)) # トレーニングデータ
print("test data: ",    model_dt.score(x_test_raw, y_test_le))  # テストデータ
#print("training data: ", model_dt.score(x_train_ss, y_train_le)) # トレーニングデータ（スケーリング後）
#print("test data: ",    model_dt.score(x_test_ss, y_test_le))  # テストデータ（スケーリング後）

# ランダムフォレストモデル
print("=== Decision tree ===")
print("training data: ", model_rf.score(x_train_raw, y_train_le)) # トレーニングデータ
print("test data: ",    model_rf.score(x_test_raw, y_test_le))  # テストデータ
#print("training data: ", model_rf.score(x_train_ss, y_train_le)) # トレーニングデータ（スケーリング後）
#print("test data: ",    model_rf.score(x_test_ss, y_test_le))  # テストデータ（スケーリング後）

# ニューラルネットワークモデル
print("=== Neural network ===")
print("training data: ", model_nn.score(x_train_ss, y_train_le)) # トレーニングデータ
print("test data: ",    model_nn.score(x_test_ss, y_test_le))  # テストデータ

### その他の評価指標

　先ほどの表を下記に再度載せています。この行列は、**混同行列（Confusion matrix）**と呼ばれています。

||予測ラベル=A<br>（ポジティブラベル）|予測ラベル=not A<br>（ネガティブラベル）|
|:---|:---|:---|
|**実ラベル=A<br>（ポジティブラベル）**|真陽性<br>True positive: TP|偽陰性<br>False negative: FN|
|**実ラベル=not A<br>（ネガティブラベル）**|偽陽性<br>False positive: FP|真陰性<br>True negative: TN|

　正解率のほかにも、評価指標はあります。ここでは、おもなものリストアップします。

- 精度 Precision  
ポジティブラベルと予測されたもののうち、実際にポジティブラベルであるサンプルの割合
$$Precision = \frac{TP}{TP + FP}$$

- False discovery rate (FDR)
ポジティブラベルと予測されたもののうち、誤って予測されたサンプルの割合。なお、ゲノム解析のひとつ、遺伝子発現解析でよく使われる指標です。
$$FDR = \frac{FP}{TP + FP} = 1 - Precision$$ 

- 検出率 Recall / 真陽性率 True positive rate / 感度 Sensitivity  
ポジティブラベルのサンプルのうち、ポジティブラベルと予測された割合
$$Recall = \frac{TP}{TP + FN}$$

- 偽陰性率 False negative rate (FNR)  
ポジティブラベルのサンプルのうち、ネガティブラベルと予測された割合
$$FNR = \frac{FP}{FP + TN} = 1 - Recall$$

- 偽陽性率 False positive rate (FPR)  
ネガティブラベルのサンプルのうち、ポジティブラベルと予測された割合
$$FPR = \frac{FP}{FP + TN}$$






　以下では、混同行列（Confusion matrix）を作るコードを紹介しています。また、精度（Precision）をscikit-learnを使って調べる方法も紹介しています。

```python
# 混同行列
from sklearn.metrics import confusion_matrix
confusion_matrix(実ラベル、予測ラベル)

# 精度
from sklearn.metrics import precision_score
precision_score(実ラベル、予測ラベル)
```

　例として、ロジスティック回帰モデルからの予測ラベルを使っています。


In [0]:
from sklearn.metrics import confusion_matrix, precision_score

# 実測値
y_train = y_train_le
y_test = y_test_le

# ロジスティック回帰モデルからの予測値
y_train_pred = model_lr.predict(x_train_ss)
y_test_pred = model_lr.predict(x_test_ss)

# トレーニングデータの混同行列
cm = confusion_matrix(y_train, y_train_pred)
pre = precision_score(y_train, y_train_pred, average=None)
print("=== training data ===")
print(cm)
print("precision=", pre)

# テストデータの混同行列
cm = confusion_matrix(y_test, y_test_pred)
pre = precision_score(y_test, y_test_pred, average=None)
print("=== test data ===")
print(cm)
print("precision=", pre)

---

## まとめ

　今回、前処理、5つの分類アルゴリズムの概要とその実装方法、モデルの評価方法を勉強しました。

　まず、前処理では、**ラベル（目的変数）の数値変換**をおこないました。

　次に、5つの分類アルゴリズムを勉強しました。それぞれのアルゴリズムは、3行のコードで実装できます。
- **ロジスティック回帰**: あるクラスに分類される確率を求める手法
```python
from sklearn.ensemble import RandomForestClassifier
model_dt = RandomForestClassifier(ハイパーパラメータ)
model_dt.fit(トレーニングデータ)
```

- **サポートベクトルマシン**: 最も近いデータまでの距離（マージン margin）が最大となる境界を作り、データを二分する方法
```python
from sklearn.svm import SVC
model_lr = SVC(ハイパーパラメータ)
model_lr.fit(トレーニングデータ)
```

- **決定木**: 説明変数に対する問い（True or False）を繰り返して、木構造の分類モデルを作る方法
```python
from sklearn.tree import DecisionTreeClassifier
model_dt = DecisionTreeClassifier(ハイパーパラメータ)
model_dt.fit(トレーニングデータ)
```

- **ランダムフォレスト**: 各決定木の予測結果を多数決して、最終的な予測値を決める手法
```python
from sklearn.ensemble import RandomForestClassifier
model_dt = RandomForestClassifier(ハイパーパラメータ)
model_dt.fit(トレーニングデータ)
```

- **ニューラルネットワーク**: 神経細胞が信号を伝達する仕組みを模した機械学習アルゴリズム
```python
from sklearn.neural_network import MLPClassifier
model_nn = MLPClassifier(ハイパーパラメータ)
model_nn.fit(トレーニングデータ)
```
　最後に、ラベルの**正解率**でそれぞれのモデルの評価をおこないました。ほかにも評価指標はあり、実際の解析では、モデルを構築する目的に合わせて、評価指標を選ぶ必要があります。